In [1]:
from pyspark.sql.types import *
#from pyspark.sql.types import sqlContext

In [2]:
import pandas as pd
import time

In [3]:
# Optimize the convertion of pandas and spark dataframes
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", 1000)

In [4]:
#pagelinkscorrect_sdf = sqlContext.read.parquet("pagelinks_corrected0spark.parquet/")
pagelinkscorrect_sdf = sqlContext.read.parquet("/home/benjamin/wikipedia/Wikipedia/pagelinks_parsed2.corrected.parquet/")

In [5]:
pagelinkscorrect_sdf.printSchema()

root
 |-- source: string (nullable = true)
 |-- fix_target: string (nullable = true)



In [6]:
pagelinkscorrect_sdf.registerTempTable('plcorrected')

In [7]:
query = """
select * from plcorrected limit 50
"""
sqlContext.sql(query).toPandas()

,source,fix_target
0,Rick_Bates,2001_Phillip_Island_V8_Supercar_round
1,Marcos_Ambrose,2001_Phillip_Island_V8_Supercar_round
2,Phillip_Island_SuperSprint,2001_Phillip_Island_V8_Supercar_round
3,2001_Shell_Championship_Series,2001_Phillip_Island_V8_Supercar_round
4,2001_Clipsal_500,2001_Phillip_Island_V8_Supercar_round
5,Alan_Jones_(racing_driver),2001_Phillip_Island_V8_Supercar_round
6,Leanne_Tander,2001_Phillip_Island_V8_Supercar_round
7,2002_Phillip_Island_V8_Supercar_round,2001_Phillip_Island_V8_Supercar_round
8,2000_FAI_1000,2001_Phillip_Island_V8_Supercar_round
9,Rick_Kelly,2001_Phillip_Island_V8_Supercar_round


### Finding duplicates

In [9]:
pagelinkscorrect_sdf.count()

10345934

In [10]:
pagelinkscorrect_sdf.distinct().count()

10242438

### Finding self-edges

In [13]:
pagelinkscorrect_sdf.filter(pagelinkscorrect_sdf['source'] == pagelinkscorrect_sdf['fix_target']).show()

+--------------------+--------------------+
|              source|          fix_target|
+--------------------+--------------------+
|  2001_in_Cape_Verde|  2001_in_Cape_Verde|
|2001–02_Croatian_...|2001–02_Croatian_...|
|2001–02_Football_...|2001–02_Football_...|
|2002_European_Aqu...|2002_European_Aqu...|
|      2002_in_Kuwait|      2002_in_Kuwait|
|2003_Beach_Soccer...|2003_Beach_Soccer...|
|2003_Arab_Unified...|2003_Arab_Unified...|
|      2005_in_Poland|      2005_in_Poland|
|2005–06_FIBA_Euro...|2005–06_FIBA_Euro...|
|    2006_Tippeligaen|    2006_Tippeligaen|
|2006_in_Croatian_...|2006_in_Croatian_...|
|Clemson_Tigers_me...|Clemson_Tigers_me...|
|2006–07_Isthmian_...|2006–07_Isthmian_...|
|2007_Grand_Prix_(...|2007_Grand_Prix_(...|
|SMU_Mustangs_foot...|SMU_Mustangs_foot...|
|2007_Shelbourne_F...|2007_Shelbourne_F...|
|  2007_in_M-1_Global|  2007_in_M-1_Global|
|2007–08_Leeds_Uni...|2007–08_Leeds_Uni...|
|2008_NCAA_Divisio...|2008_NCAA_Divisio...|
|2009_bombing_of_I...|2009_bombi

In [14]:
pagelinkscorrect_sdf.filter(pagelinkscorrect_sdf['source'] == pagelinkscorrect_sdf['fix_target']).count()

8661

In [19]:
pagelinkscorrect_sdf.filter(pagelinkscorrect_sdf['fix_target'] == None).show()

+------+----------+
|source|fix_target|
+------+----------+
+------+----------+



In [ ]:
pagelinkscorrect_sdf